In [2]:
import polars as pl 
import gzip 
from lxml import etree
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 
import pyarrow.parquet as pq
import pyarrow as pa



def timtim(fonction):
    def wrapper(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        temps_execution = fin - debut
        print(f"La fonction {fonction.__name__} a pris {temps_execution:.5f} secondes pour s'exécuter.")
        return resultat
    return wrapper

def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def recherche_id_fichier(chemin_parquet) :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id


def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier


def nettoyage_adaptatif(dataframe : pd.DataFrame) -> pd.DataFrame :
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in dataframe.columns : 
  if col in ['CaracSup', 'CaracSup'] : 
   dataframe[col] = dataframe[col].astype(str) 
  else :
   dataframe[col] = dataframe[col].apply(nettoyage)
 dataframe_propre = dataframe.reset_index(drop=True)
 return dataframe_propre

In [3]:
DOSSIER_PARQUETS_INDIVIDUELS = './fichiers/todo_parquet/'

In [10]:
fichier_concatene = "grosparquet.parquet"
table_polars = pl.read_parquet(fichier_concatene)
table_test = table_polars.__copy__

In [39]:
import pandas as pd 

table_pandas = pd.read_parquet(fichier_concatene)



In [41]:
table_pandas['CaracSup']

0                 {'@Code': 'BudgetHorsRAR', '@V': '3000'}
1                  {'@Code': 'BudgetHorsRAR', '@V': '350'}
2                 {'@Code': 'BudgetHorsRAR', '@V': '5950'}
3                 {'@Code': 'BudgetHorsRAR', '@V': '3800'}
4                 {'@Code': 'BudgetHorsRAR', '@V': '8500'}
                               ...                        
59870    [{'@Code': 'ProdChaRat', '@V': '1172.27'}, {'@...
59871    [{'@Code': 'ProdChaRat', '@V': '8723.85'}, {'@...
59872                                                  nan
59873                                                  nan
59874         {'@Code': 'BudgetHorsRAR', '@V': '47340.00'}
Name: MtSup, Length: 59875, dtype: object

In [56]:
table_pandas.iloc[59870]['MtSup']

"[{'@Code': 'ProdChaRat', '@V': '1172.27'}, {'@Code': 'BudgetHorsRAR', '@V': '5600.00'}]"

In [52]:

import pandas as pd

# Création d'un DataFrame de test
data = [{'@Code': 'BudgetHorsRAR', '@V': '3000'}, {'@Code': 'BudgetHorsRAR', '@V': '350'},
         {'@Code': 'BudgetHorsRAR', '@V': '5950'}, {'@Code': 'BudgetHorsRAR', '@V': '3800'},
         {'@Code': 'BudgetHorsRAR', '@V': '8500'}, [{'@Code': 'ProdChaRat', '@V': '1172.27'}, {'@Code': 'BudgetHorsRAR', '@V': '5600.00'}] ]

df = pd.DataFrame(data)

# Affichage du DataFrame d'origine
df

,@Code,@V
0,BudgetHorsRAR,3000
1,BudgetHorsRAR,350
2,BudgetHorsRAR,5950
3,BudgetHorsRAR,3800
4,BudgetHorsRAR,8500


Option possible : On va tester d'intégrer les fonctions qui transforment MtSup et CaracSup dans la task1 

In [125]:
def task_2(dossier, nom_parquet, schema_cust) :
 liste_parquet_t = []
 for parquet in glob.glob(os.path.join(dossier, "*")) : 
  liste_parquet_t.append(parquet)

 schema = schema_cust
 #schema = pq.ParquetFile(liste_parquet_t[0]).schema_arrow
 with pq.ParquetWriter(f"{nom_parquet}.parquet", schema=schema) as writer:
    for file in liste_parquet_t:
        table = pq.read_table(file, schema=schema)
        writer.write_table(table)

In [4]:
def _transformation_mtsup_original(lignes_budget: dict) -> dict:
 '''Grâce aux spaghettis, il y a deux types de MtSup dans les fichiers,
       des dict et des listes de dict, permet de gérer les deux cas
    '''
 for i in lignes_budget:
  type_mtsup = i.get('MtSup')  # Permet de connaitre le type de MtSup
  if type_mtsup is not None:  # Vérifie si la clé 'MtSup' existe
    if isinstance(type_mtsup, dict):
      dict_mtsup = type_mtsup
      i['MtSup_1_Lib'] = {'@V': dict_mtsup.get('@Code', '')}
      i['MtSup_1_Val'] = {'@V': dict_mtsup.get('@V', '')}
    elif isinstance(type_mtsup, list):
      dict_mtsup = i['MtSup']
      mtsup_propre = {}
      for z, entry in enumerate(dict_mtsup, start=1):
        code = f'MtSup_{z}_Lib'
        valeur = f'MtSup_{z}_Val'
        mtsup_propre[code] = entry.get('@Code', '')
        mtsup_propre[valeur] = entry.get('@V', '')
        i.update(mtsup_propre)
        
 return lignes_budget
   
def _transformation_caracsup_original(lignes_budget: dict) -> dict:
 '''Grâce aux spaghettis, il y a deux types de CaracSup dans les fichiers,
       des dict et des listes de dict, permet de gérer les deux cas
    '''
 for i in lignes_budget:
  type_caracsup = i.get('CaracSup')  # Permet de connaitre le type de CaracSup
  if type_caracsup is not None:  # Vérifie si la clé 'CaracSup' existe
    if isinstance(type_caracsup, dict):
      dict_caracsup = type_caracsup
      i['CaracSup_1_Lib'] = {'@V': dict_caracsup.get('@Code', '')}
      i['CaracSup_1_Val'] = {'@V': dict_caracsup.get('@V', '')}
    elif isinstance(type_caracsup, list):
      dict_caracsup = i['CaracSup']
      caracsup_propre = {}
      for z, entry in enumerate(dict_caracsup, start=1):
        code = f'CaracSup_{z}_Lib'
        valeur = f'CaracSup_{z}_Val'
        caracsup_propre[code] = entry.get('@Code', '')
        caracsup_propre[valeur] = entry.get('@V', '')
        i.update(caracsup_propre)
        
 return lignes_budget

In [88]:
def Extraction_xml_to_parquet_v31(chemin) : 
 ''' Traite les XML et les envoies sous format parquet,
   le nettoyage des @V est fait à cette étape'''
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
   docbase = _transformation_caracsup_original(docbase)
   docbase = _transformation_mtsup_original(docbase)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_adaptatif(df_base)
   df_propre.to_parquet(f'./fichiers/todo_parquet/{id_fichier}_pyarrow', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(e)
    continue  


DOSSIER_TODO = './fichiers/todo_xml/'


In [6]:
chemin_test_solo = './fichiers/todo_xml/612019.xml.gz'

In [7]:
id_612019 = '612019'
dico_612019 = parse_fichier(chemin_test_solo)
dict_612019 = dico_612019['DocumentBudgetaire']['Budget']['LigneBudget']
dict_612019 = _transformation_mtsup_original(dict_612019)
dict_612019 = _transformation_caracsup_original(dict_612019)
df_612019 = pd.DataFrame(dict_612019)
df_612019.head(5)

FileNotFoundError: [Errno 2] No such file or directory: './fichiers/todo_xml/612019.xml.gz'

In [89]:
Extraction_xml_to_parquet_v31(DOSSIER_TODO)

Erreur fichier ./fichiers/todo_xml\619027.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/todo_xml\628857.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/todo_xml\682109.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/todo_xml\722795.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/todo_xml\730073.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/todo_xml\748147.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/todo_xml\756300.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/todo_xml\775775.xml.gz, extraction impossible
no element found: line 1, column 0
Erreur fichier ./fichiers/todo_xml\807134.xml.gz, extraction impossible
'str' object has no attribute 'get'
Erreur fichier ./fichiers/tod

In [94]:
task_2(DOSSIER_PARQUETS_INDIVIDUELS, 'test_mtsup_prefail')

Nombre d'erreur (hors 775775 qui est corrompu) : 13

In [8]:
idfail_619027 = '619027'
parsefail_619027 = parse_fichier('..//fichiers/todo_xml/619027.xml.gz')  
dictfail_619027 = parsefail_619027['DocumentBudgetaire']['Budget']['LigneBudget']
dfprefail_619027 = pd.DataFrame(dictfail_619027)
test_619027 = _transformation_mtsup_original(dictfail_619027)

AttributeError: 'str' object has no attribute 'get'

In [93]:
dfprefail_619027.columns

Index(['Nature', 'ArtSpe', 'CodRD', 'MtReal', 'MtRAR3112', 'OpBudg'], dtype='object')

On peut imaginer que c'est pareil pour les autres, on a déjà eu ça avant, il faut juste modifier la vérification, voir que ça existe. 

In [95]:
parquet_concat = 'test_mtsup_prefail.parquet'
table_test_polars = pl.read_parquet(fichier_concatene)
table_test_polars.columns

['Nature',
 'LibCpte',
 'ContNat',
 'ArtSpe',
 'CodRD',
 'MtBudgPrec',
 'MtRARPrec',
 'MtPropNouv',
 'MtPrev',
 'CredOuv',
 'MtReal',
 'MtRAR3112',
 'OpBudg',
 'MtSup',
 'CaracSup',
 'DteStr',
 'LibellePoste',
 'IdPost',
 'LibelleColl',
 'IdColl',
 'NatCEPL',
 'Departement',
 'Id_Fichier']

Il est temps de créer un schema

In [120]:
df_tt = pd.read_parquet('./fichiers/todo_parquet/766039_pyarrow')
df_tt.columns

Index(['Nature', 'LibCpte', 'ContNat', 'ArtSpe', 'ContFon', 'CodRD',
       'MtRARPrec', 'CredOuv', 'MtReal', 'MtRAR3112', 'OpBudg', 'CaracSup',
       'DteStr', 'LibellePoste', 'IdPost', 'LibelleColl', 'IdColl', 'NatCEPL',
       'Id_Fichier', 'CaracSup_1_Lib', 'CaracSup_1_Val', 'Fonction', 'MtSup',
       'CaracSup_2_Lib', 'CaracSup_2_Val', 'MtSup_1_Lib', 'MtSup_1_Val',
       'TypOpBudg', 'OpeCpteTiers', 'CaracSup_3_Lib', 'CaracSup_3_Val'],
      dtype='object')

In [122]:
df_tt['TypOpBudg'].unique()

array(['nan', '2', '1'], dtype=object)

In [131]:
custom_schema = pa.schema([
        ('Nature', pa.string()),
        ('Fonction', pa.string()),
        ('LibCpte', pa.string()),
        ('ContNat', pa.string()),
        ('ArtSpe', pa.string()), #Doit être corrigé en bool,
        ('ContFon', pa.string()),
        ('CodRD', pa.string()), #Val D ou R,
        ('MtBudgPrec', pa.string()), #float32 #nullable = True
        ('MtRARPrec', pa.string()), #float32
        ('MtPropNouv', pa.string()), #float32
        ('MtPrev', pa.string()), #float32
        ('CredOuv', pa.string()), #int32
        ('MtReal', pa.string()), #float32
        ('MtRAR3112', pa.string()), #float32
        ('OpBudg', pa.string()),
        ('MtSup', pa.string()),
        ('MtSup_1_Lib', pa.string()),
        ('MtSup_1_Val', pa.string()),
        ('MtSup_2_Lib', pa.string()),
        ('MtSup_2_Val', pa.string()),
        ('MtSup_3_Lib', pa.string()),
        ('MtSup_3_Val', pa.string()),
        ('MtSup_4_Lib', pa.string()),
        ('MtSup_4_Val', pa.string()),
        ('MtSup_5_Lib', pa.string()),
        ('MtSup_5_Val', pa.string()),
        ('MtSup_6_Lib', pa.string()),
        ('MtSup_6_Val', pa.string()),
        ('MtSup_7_Lib', pa.string()),
        ('MtSup_7_Val', pa.string()),
        ('MtSup_8_Lib', pa.string()),
        ('MtSup_8_Val', pa.string()),
        ('MtSup_9_Lib', pa.string()),
        ('MtSup_9_Val', pa.string()),
        ('MtSup_10_Lib', pa.string()),
        ('MtSup_10_Val', pa.string()), #10
        ('CaracSup_1_Lib', pa.string()),
        ('CaracSup_1_Val', pa.string()),
        ('CaracSup_2_Lib', pa.string()),
        ('CaracSup_2_Val', pa.string()),
        ('CaracSup_3_Lib', pa.string()),
        ('CaracSup_3_Val', pa.string()),
        ('CaracSup_4_Lib', pa.string()),
        ('CaracSup_4_Val', pa.string()),
        ('CaracSup_5_Lib', pa.string()),
        ('CaracSup_5_Val', pa.string()),
        ('CaracSup_6_Lib', pa.string()),
        ('CaracSup_6_Val', pa.string()),
        ('CaracSup_7_Lib', pa.string()), #7
        ('CaracSup_7_Val', pa.string()),
        ('TypOpBudg', pa.string()), #des 2 et des 1
        ('OpeCpteTiers', pa.string()),
        ('DteStr', pa.date32()),
        ('LibellePoste', pa.string()),
        ('IdPost', pa.string()),
        ('LibelleColl', pa.string()),
        ('IdColl', pa.string()),
        ('NatCEPL', pa.string()),
        ('Departement', pa.string()), #On oublie pas les 2A,
        ('Id_Fichier', pa.int64())
    ])

In [112]:
table_polars.head()

Nature,LibCpte,ContNat,ArtSpe,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,MtSup,CaracSup,DteStr,LibellePoste,IdPost,LibelleColl,IdColl,NatCEPL,Departement,Id_Fichier
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""60623""","""Alimentation p…","""011""","""false""","""D""","""6000""","""0""","""0""","""0""","""3000""","""2631.57""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""60628""","""Autres fournit…","""011""","""false""","""D""","""750""","""0""","""0""","""0""","""350""","""303.24""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""6063""","""{'@V': ""Fourni…","""011""","""false""","""D""","""16580""","""0""","""0""","""0""","""5950""","""5185.9""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""6065""","""Livres, disque…","""011""","""false""","""D""","""6050""","""0""","""0""","""0""","""3800""","""5082.13""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""6067""","""Fournitures sc…","""011""","""false""","""D""","""17774""","""0""","""0""","""0""","""8500""","""6874.29""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""


In [111]:
table_polars['ArtSpe'].unique()

ArtSpe
str
"""true"""
"""false"""
null
"""0"""


In [119]:
table_polars['OpBudg'].unique()

OpBudg
str
"""1"""
"""0"""


In [110]:
table_polars.head()

Nature,LibCpte,ContNat,ArtSpe,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,MtSup,CaracSup,DteStr,LibellePoste,IdPost,LibelleColl,IdColl,NatCEPL,Departement,Id_Fichier
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""60623""","""Alimentation p…","""011""","""false""","""D""","""6000""","""0""","""0""","""0""","""3000""","""2631.57""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""60628""","""Autres fournit…","""011""","""false""","""D""","""750""","""0""","""0""","""0""","""350""","""303.24""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""6063""","""{'@V': ""Fourni…","""011""","""false""","""D""","""16580""","""0""","""0""","""0""","""5950""","""5185.9""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""6065""","""Livres, disque…","""011""","""false""","""D""","""6050""","""0""","""0""","""0""","""3800""","""5082.13""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""
"""6067""","""Fournitures sc…","""011""","""false""","""D""","""17774""","""0""","""0""","""0""","""8500""","""6874.29""","""0""","""0""","""{'@Code': 'Bud…","""{'@Code': 'Pro…","""2019-01-01""","""CENTRE DES FIN…","""078027""","""Caisse des Eco…","""26780487000017…","""Commune""","""78""","""612019"""


In [132]:
task_2(DOSSIER_PARQUETS_INDIVIDUELS, 'parquet_schema', schema_cust= custom_schema)

In [135]:
polars_schema_table = pl.read_parquet("./parquet_schema.parquet")
polars_schema_table.head(5)

Nature,Fonction,LibCpte,ContNat,ArtSpe,ContFon,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,MtSup,MtSup_1_Lib,MtSup_1_Val,MtSup_2_Lib,MtSup_2_Val,MtSup_3_Lib,MtSup_3_Val,MtSup_4_Lib,MtSup_4_Val,MtSup_5_Lib,MtSup_5_Val,MtSup_6_Lib,MtSup_6_Val,MtSup_7_Lib,MtSup_7_Val,MtSup_8_Lib,MtSup_8_Val,MtSup_9_Lib,MtSup_9_Val,MtSup_10_Lib,MtSup_10_Val,CaracSup_1_Lib,CaracSup_1_Val,CaracSup_2_Lib,CaracSup_2_Val,CaracSup_3_Lib,CaracSup_3_Val,CaracSup_4_Lib,CaracSup_4_Val,CaracSup_5_Lib,CaracSup_5_Val,CaracSup_6_Lib,CaracSup_6_Val,CaracSup_7_Lib,CaracSup_7_Val,TypOpBudg,OpeCpteTiers,DteStr,LibellePoste,IdPost,LibelleColl,IdColl,NatCEPL,Departement,Id_Fichier
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,str,str,str,str,str,str,i64
"""60632""",null,"""nan""","""011""","""false""",null,"""D""","""11962.00""","""nan""","""19335.00""","""19335.00""","""19335.00""",null,null,"""0""","""{'@V': '11962.…","""BudgetHorsRAR""","""11962.00""","""nan""","""nan""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Section""","""F""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,"""nan""",null,2020-08-01,"""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""",612013
"""6332""",null,"""nan""","""012""","""false""",null,"""D""","""450.00""","""nan""","""500.00""","""500.00""","""500.00""",null,null,"""0""","""{'@V': '450.00…","""BudgetHorsRAR""","""450.00""","""nan""","""nan""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Section""","""F""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,"""nan""",null,2020-08-01,"""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""",612013
"""6411""",null,"""nan""","""012""","""false""",null,"""D""","""365000.00""","""nan""","""352000.00""","""352000.00""","""352000.00""",null,null,"""0""","""{'@V': '365000…","""BudgetHorsRAR""","""365000.00""","""nan""","""nan""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Section""","""F""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,"""nan""",null,2020-08-01,"""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""",612013
"""74834""",null,"""nan""","""nan""","""false""",null,"""R""","""4104.00""","""nan""","""4084.00""","""4084.00""","""4084.00""",null,null,"""0""","""{'@V': '4104.0…","""BudgetHorsRAR""","""4104.00""","""nan""","""nan""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Section""","""F""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,"""nan""",null,2020-08-01,"""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""",612013
"""6331""",null,"""nan""","""012""","""false""",null,"""D""","""4250.00""","""nan""","""5200.00""","""5200.00""","""5200.00""",null,null,"""0""","""{'@V': '4250.0…","""BudgetHorsRAR""","""4250.00""","""nan""","""nan""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Section""","""F""","""nan""","""nan""",null,null,null,null,null,null,null,null,null,null,"""nan""",null,2020-08-01,"""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""",612013


In [151]:
polars_schema_table['MtSup_10_Lib'].unique()

MtSup_10_Lib
str
"""BudgetHorsRAR"""
null
"""nan"""


In [152]:
df_filtered = polars_schema_table.filter((pl.col('MtSup_2_Lib').is_not_null()) & (pl.col('MtSup_2_Lib') != ''))


In [153]:
print(df_filtered)


shape: (1_887_271, 60)
┌────────┬──────────┬─────────┬─────────┬───┬────────────────┬─────────┬─────────────┬────────────┐
│ Nature ┆ Fonction ┆ LibCpte ┆ ContNat ┆ … ┆ IdColl         ┆ NatCEPL ┆ Departement ┆ Id_Fichier │
│ ---    ┆ ---      ┆ ---     ┆ ---     ┆   ┆ ---            ┆ ---     ┆ ---         ┆ ---        │
│ str    ┆ str      ┆ str     ┆ str     ┆   ┆ str            ┆ str     ┆ str         ┆ i64        │
╞════════╪══════════╪═════════╪═════════╪═══╪════════════════╪═════════╪═════════════╪════════════╡
│ 60632  ┆ null     ┆ nan     ┆ 011     ┆ … ┆ 21240002200019 ┆ Commune ┆ 24          ┆ 612013     │
│ 6332   ┆ null     ┆ nan     ┆ 012     ┆ … ┆ 21240002200019 ┆ Commune ┆ 24          ┆ 612013     │
│ 6411   ┆ null     ┆ nan     ┆ 012     ┆ … ┆ 21240002200019 ┆ Commune ┆ 24          ┆ 612013     │
│ 74834  ┆ null     ┆ nan     ┆ nan     ┆ … ┆ 21240002200019 ┆ Commune ┆ 24          ┆ 612013     │
│ …      ┆ …        ┆ …       ┆ …       ┆ … ┆ …              ┆ …       ┆ …   